In [60]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

In [61]:
START_DATE = "2015-01-01"
END_DATE = "2026-01-01"
YF_TICKERS = ["NVDA", "DIS", "SHEL.L", "BND", "TLT", "GC=F", "CL=F", "GLD", "GBPUSD=X", "BTC-USD"]
MACRO_TICKERS = ["CPIAUCSL", "DGS10", "INDPRO", "UNRATE"]
TIME_INTERVAL = '1mo'
VOL_WINDOW = 30
MOM_WINDOW = 90 
MA_WINDOW = 20 

asset_data = yf.download(YF_TICKERS, start = START_DATE, end = END_DATE, interval = TIME_INTERVAL)['Close'].fillna(0.0)

macro_data = web.DataReader(MACRO_TICKERS, "fred", START_DATE, END_DATE).ffill().bfill()
macro_data['INDPRO_Growth_%'] = macro_data['INDPRO'].pct_change(periods=12).fillna(0.0) * 100
macro_data['Inflation_%'] = macro_data['CPIAUCSL'].pct_change(periods=12).fillna(0.0) * 100
macro_data = macro_data.drop(columns=['CPIAUCSL', 'INDPRO'])
macro_data = macro_data.rename(columns={
    'DGS10': '10Y_Bond_Yield_%',
    'UNRATE': 'Unemployment_Rate_%'
})

df = asset_data.join(macro_data)


/var/folders/nn/rjk_sthj67g1rcv7qxd8_14m0000gn/T/ipykernel_94198/378993610.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  asset_data = yf.download(YF_TICKERS, start = START_DATE, end = END_DATE, interval = TIME_INTERVAL)['Close'].fillna(0.0)
[*********************100%***********************]  10 of 10 completed


In [62]:
df = df.sort_index().dropna()

asset_tickers = ["NVDA", "DIS", "SHEL.L", "BND", "TLT", "GC=F", "CL=F", "GLD", "GBPUSD=X", "BTC-USD"]
macro_vars = ['10Y_Bond_Yield_%', 'Unemployment_Rate_%', 'INDPRO_Growth_%', 'Inflation_%']

asset_returns = df[asset_tickers].pct_change().dropna()

monthly_risk_free_rate = (df['10Y_Bond_Yield_%'].dropna() / 100) / 12

excess_returns_df = asset_returns.subtract(monthly_risk_free_rate, axis=0).dropna()

print("\nStep 2: Excess Returns DataFrame (Head)")
print(excess_returns_df.head())


scaler = StandardScaler()
scaled_returns = scaler.fit_transform(excess_returns_df)

pca = PCA(n_components=None) 
pca.fit(scaled_returns)

cumulative_variance = np.cumsum(pca.explained_variance_ratio_)

k = np.argmax(cumulative_variance >= 0.80) + 1
print(f"\nStep 3 & 4: Retaining {k} principal components to explain >80% variance.")

pca_final = PCA(n_components=k)
principal_components = pca_final.fit_transform(scaled_returns)

pc_df = pd.DataFrame(principal_components, columns=[f'PC{i+1}' for i in range(k)], index=excess_returns_df.index)

lagged_macro_data = df[macro_vars].shift(1).dropna()

pc_df_aligned = pc_df.loc[lagged_macro_data.index]
lagged_macro_data_aligned = lagged_macro_data.loc[pc_df_aligned.index]


print(f"\nStep 5: Running OLS Regressions of PCs on lagged macro variables...")
regression_summaries = {}

X = sm.add_constant(lagged_macro_data_aligned)

for pc_col in pc_df_aligned.columns:
    Y = pc_df_aligned[pc_col]
    model = sm.OLS(Y, X).fit()
    regression_summaries[pc_col] = model
    print(f"\n--- Regression Summary for {pc_col} ---")
    print(model.summary())

print("\nStep 6: Interpretation (requires manual review of summaries above)")
print("Examine the p-values and coefficients in the summaries above.")
print("A PC with a statistically significant coefficient (p < 0.05) on 'Inflation_%' is likely the 'Inflation Factor'.")
print("A PC with a significant negative coefficient on 'Unemployment_Rate_%' (and positive on 'INDPRO_Growth_%') likely corresponds to the 'Growth Factor'.")

plt.figure(figsize=(8, 5))
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_, marker='o', linestyle='--')
plt.xlabel('Number of Components')
plt.ylabel('Proportion of Variance Explained')
plt.title('Scree Plot (Variance Explained by PCs)')
plt.grid(True)
plt.show()

loadings = pd.DataFrame(pca_final.components_.T, columns=[f'PC{i+1}' for i in range(k)], index=asset_tickers)
plt.figure(figsize=(10, 8))
sns.heatmap(loadings, annot=True, cmap='coolwarm', center=0)
plt.title('Heatmap of Asset Loadings on Principal Components')
plt.xlabel('Principal Components')
plt.ylabel('Assets')
plt.show()



Step 2: Excess Returns DataFrame (Head)
                NVDA       DIS    SHEL.L       BND       TLT      GC=F  \
Date                                                                     
2015-02-01  0.147558  0.142839  0.048154 -0.016456 -0.064678 -1.001400   
2015-04-01  0.059120  0.034956  0.024089 -0.005133 -0.035949       inf   
2015-05-01 -0.004920  0.013410 -0.058334 -0.006362 -0.025452  0.004153   
2015-06-01 -0.088898  0.032333 -0.082232 -0.013045 -0.042644 -0.016875   
2015-07-01 -0.009981  0.049315  0.027922  0.006777  0.043387 -0.067411   

                CL=F       GLD  GBPUSD=X   BTC-USD  
Date                                                
2015-02-01 -1.001400 -0.060452  0.023023  0.167819  
2015-04-01       inf -0.003230  0.040156 -0.034639  
2015-05-01  0.009469  0.003785 -0.008752 -0.026984  
2015-06-01 -0.015589 -0.016987  0.025124  0.141022  
2015-07-01 -0.209693 -0.068235 -0.010139  0.079998  


ValueError: Input X contains infinity or a value too large for dtype('float64').